Установка всех зависимостей

In [ ]:
pip install -r requirements.txt

Импорт всех библиотек

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

Загрузка датасета 

In [ ]:
df = pd.read_csv("data.csv", sep=";")
dataset = Dataset.from_pandas(df)
train_dataset = dataset.train_test_split(0.15)

Загрузка модели и токенизатора

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'DeepPavlov/rubert-base-cased-sentence', num_labels=2)
tokenizer = BertTokenizer.from_pretrained(
    'DeepPavlov/rubert-base-cased-sentence')

Фукнция токенизации и её применение к датасету

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=512)


# Применение токенизации
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_train_dataset = dataset.map(tokenize_function, batched=True)


Настройка дообучения модели

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=4,
    per_device_train_batch_size=4,
    save_steps=10_000,
    learning_rate=5e-5,
    save_total_limit=2,
    logging_dir=str("./logs"),
)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

Дообучение модели

In [ ]:
trainer.train()

Сохранение дообученной модели

In [ ]:
model.save_pretrained("result/model")
tokenizer.save_pretrained("result/tokenizer")